## Análisis de Dispersión y Tests de Distribución

<div align="center">

# **Modelacion**

</div>

## Librerias

## Data

In [14]:
siniestros_ppd <- read.csv("../data/processed/Siniestros_ppd.csv")
siniestros_pph <- read.csv("../data/processed/Siniestros_pph.csv")
siniestros_pth <- read.csv("../data/processed/Siniestros_pth.csv")
siniestros_rc <- read.csv("../data/processed/Siniestros_rc.csv")
polizas_ppd <- read.csv("../data/processed/polizas_ppd.csv")
polizas_pph <- read.csv("../data/processed/polizas_pph.csv")
polizas_pth <- read.csv("../data/processed/polizas_pth.csv")
polizas_rc <- read.csv("../data/processed/polizas_rc.csv")

In [15]:
siniestros_pph$Año <- NULL
siniestros_pph$Mes <- NULL
siniestros_pph$Día <- NULL
siniestros_pph$Periodo <- NULL

In [16]:
siniestros_ppd$Año <- NULL
siniestros_ppd$Mes <- NULL
siniestros_ppd$Día <- NULL
siniestros_ppd$Periodo <- NULL

In [17]:
siniestros_pth$Año <- NULL
siniestros_pth$Mes <- NULL
siniestros_pth$Día <- NULL
siniestros_pth$Periodo <- NULL

In [18]:
siniestros_rc$Año <- NULL
siniestros_rc$Mes <- NULL
siniestros_rc$Día <- NULL
siniestros_rc$Periodo <- NULL

## Merge de las tablas

In [19]:
ppd <- merge(polizas_ppd, siniestros_ppd, by.x = "Fecha", by.y = "Fecha_siniestro")
pph <- merge(polizas_pph, siniestros_pph, by.x = "Fecha", by.y = "Fecha_siniestro")
pth <- merge(polizas_pth, siniestros_pth, by.x = "Fecha", by.y = "Fecha_siniestro")
rc <- merge(polizas_rc, siniestros_rc, by.x = "Fecha", by.y = "Fecha_siniestro")

In [20]:
ppd$X <- NULL
pph$X <- NULL
pth$X <- NULL
rc$X <- NULL

In [21]:
colnames(ppd)

[1] "Fecha"                            "Numero_polizas"                  
[3] "Suma_primas"                      "Exposicion"                      
[5] "Cantidad_siniestros"              "Valor_siniestro_incurrido_update"

## Modelo

### PPD

In [22]:
l_pph <- sum(pph$Cantidad_siniestros)/sum(pph$Exposicion)
l_ppd <- sum(ppd$Cantidad_siniestros)/sum(ppd$Exposicion)
l_pth <- sum(pth$Cantidad_siniestros)/sum(pth$Exposicion)
l_rc <- sum(rc$Cantidad_siniestros)/sum(rc$Exposicion)

In [23]:
l_pph
l_ppd
l_pth
l_rc

[1] 3.533649e-05

[1] 0.000682542

[1] 3.412524e-05

[1] 0.0001705156

In [24]:
E_pph <- l_pph * sum(pph$Exposicion)
E_ppd <- l_ppd * sum(ppd$Exposicion)
E_pth <- l_pth * sum(pth$Exposicion)
E_rc <- l_rc * sum(rc$Exposicion)

In [25]:
E_pph
E_ppd
E_pth
E_rc

[1] 740

[1] 16516

[1] 634

[1] 3538

In [26]:
# Función para calcular estadísticos de dispersión y realizar tests
calcular_dispersion_y_tests <- function(datos_siniestros, nombre_cobertura) {
  
  # Extraer los siniestros diarios (N_t,j)
  N_t_j <- datos_siniestros$Cantidad_siniestros
  T <- length(N_t_j)  # T = 365 días
  
  # Calcular estadísticos muestrales
  media_muestral <- mean(N_t_j)
  varianza_muestral <- var(N_t_j)
  
  # Calcular D_j (dispersión muestral)
  D_j <- varianza_muestral / media_muestral
  
  # Test de Cameron-Trivedi para sobredispersión
  # H0: Poisson, H1: Binomial Negativa
  t_cameron_trivedi <- (varianza_muestral - media_muestral) / 
                       (sqrt(2) * media_muestral / sqrt(T))
  p_value_cameron <- 2 * (1 - pnorm(abs(t_cameron_trivedi)))
  
  # Test de equidispersión (Chi-cuadrado)
  # H0: D_j = 1 (Poisson), H1: D_j ≠ 1
  chi_cuadrado <- (T - 1) * D_j
  p_value_chi <- 1 - pchisq(chi_cuadrado, df = T - 1)
  
  # Test de Dean para sobredispersión
  # H0: Equidispersión, H1: Sobredispersión
  z_dean <- (varianza_muestral - media_muestral) / sqrt(2 * media_muestral / (T - 1))
  p_value_dean <- 1 - pnorm(z_dean)
  
  # Determinar distribución recomendada
  if (D_j >= 0.8 && D_j <= 1.2) {
    distribucion <- "Poisson"
  } else if (D_j > 1.5) {
    distribucion <- "Binomial Negativa"
  } else if (D_j < 0.8) {
    distribucion <- "Subdispersa"
  } else {
    distribucion <- "Borderline - revisar tests"
  }
  
  # Resultados
  resultados <- list(
    cobertura = nombre_cobertura,
    media_muestral = media_muestral,
    varianza_muestral = varianza_muestral,
    D_j = D_j,
    t_cameron_trivedi = t_cameron_trivedi,
    p_value_cameron = p_value_cameron,
    chi_cuadrado = chi_cuadrado,
    p_value_chi = p_value_chi,
    z_dean = z_dean,
    p_value_dean = p_value_dean,
    distribucion_recomendada = distribucion
  )
  
  return(resultados)
}

In [27]:
# Aplicar análisis de dispersión a cada cobertura
resultados_ppd <- calcular_dispersion_y_tests(siniestros_ppd, "PPD")
resultados_pph <- calcular_dispersion_y_tests(siniestros_pph, "PPH") 
resultados_pth <- calcular_dispersion_y_tests(siniestros_pth, "PTH")
resultados_rc <- calcular_dispersion_y_tests(siniestros_rc, "RC")

# Lista con todos los resultados
todos_resultados <- list(resultados_ppd, resultados_pph, resultados_pth, resultados_rc)

In [28]:
# Función para mostrar resultados de manera organizada
mostrar_resultados <- function(resultado) {
  cat("==============================================\n")
  cat("COBERTURA:", resultado$cobertura, "\n")
  cat("==============================================\n")
  cat("Media muestral (N̄_t,j):", round(resultado$media_muestral, 4), "\n")
  cat("Varianza muestral (s²_j):", round(resultado$varianza_muestral, 4), "\n")
  cat("Dispersión D_j:", round(resultado$D_j, 4), "\n")
  cat("\n--- TESTS ESTADÍSTICOS ---\n")
  cat("Test Cameron-Trivedi:\n")
  cat("  Estadístico t:", round(resultado$t_cameron_trivedi, 4), "\n")
  cat("  p-valor:", round(resultado$p_value_cameron, 4), "\n")
  cat("  Interpretación:", ifelse(resultado$p_value_cameron < 0.05, 
                                 "Rechaza Poisson (hay sobredispersión)", 
                                 "No rechaza Poisson"), "\n")
  cat("\nTest Chi-cuadrado (equidispersión):\n")
  cat("  Estadístico χ²:", round(resultado$chi_cuadrado, 4), "\n")
  cat("  p-valor:", round(resultado$p_value_chi, 4), "\n")
  cat("  Interpretación:", ifelse(resultado$p_value_chi < 0.05, 
                                 "Rechaza equidispersión", 
                                 "No rechaza equidispersión"), "\n")
  cat("\nTest de Dean:\n")
  cat("  Estadístico Z:", round(resultado$z_dean, 4), "\n")
  cat("  p-valor:", round(resultado$p_value_dean, 4), "\n")
  cat("  Interpretación:", ifelse(resultado$p_value_dean < 0.05, 
                                 "Hay sobredispersión significativa", 
                                 "No hay sobredispersión significativa"), "\n")
  cat("\n--- RECOMENDACIÓN ---\n")
  cat("Distribución recomendada:", resultado$distribucion_recomendada, "\n")
  cat("==============================================\n\n")
}

# Mostrar resultados para cada cobertura
for (resultado in todos_resultados) {
  mostrar_resultados(resultado)
}

COBERTURA: PPD 
Media muestral (N̄_t,j): 45.2493 
Varianza muestral (s²_j): 60.4404 
Dispersión D_j: 1.3357 

--- TESTS ESTADÍSTICOS ---
Test Cameron-Trivedi:
  Estadístico t: 4.5353 
  p-valor: 0 
  Interpretación: Rechaza Poisson (hay sobredispersión) 

Test Chi-cuadrado (equidispersión):
  Estadístico χ²: 486.2021 
  p-valor: 0 
  Interpretación: Rechaza equidispersión 

Test de Dean:
  Estadístico Z: 30.4663 
  p-valor: 0 
  Interpretación: Hay sobredispersión significativa 

--- RECOMENDACIÓN ---
Distribución recomendada: Borderline - revisar tests 

COBERTURA: PPH 
Media muestral (N̄_t,j): 2.3492 
Varianza muestral (s²_j): 1.5974 
Dispersión D_j: 0.68 

--- TESTS ESTADÍSTICOS ---
Test Cameron-Trivedi:
  Estadístico t: -4.0162 
  p-valor: 1e-04 
  Interpretación: Rechaza Poisson (hay sobredispersión) 

Test Chi-cuadrado (equidispersión):
  Estadístico χ²: 213.5135 
  p-valor: 1 
  Interpretación: No rechaza equidispersión 

Test de Dean:
  Estadístico Z: -6.1459 
  p-valor: 1 
  I

In [29]:
# Crear tabla resumida de resultados
crear_tabla_resumen <- function(todos_resultados) {
  
  # Extraer datos principales
  coberturas <- sapply(todos_resultados, function(x) x$cobertura)
  D_j_valores <- sapply(todos_resultados, function(x) round(x$D_j, 4))
  distribuciones <- sapply(todos_resultados, function(x) x$distribucion_recomendada)
  p_cameron <- sapply(todos_resultados, function(x) round(x$p_value_cameron, 4))
  
  # Crear dataframe
  tabla_resumen <- data.frame(
    Cobertura = coberturas,
    D_j = D_j_valores,
    P_valor_Cameron_Trivedi = p_cameron,
    Distribucion_Recomendada = distribuciones,
    stringsAsFactors = FALSE
  )
  
  return(tabla_resumen)
}

# Crear y mostrar tabla resumen
tabla_final <- crear_tabla_resumen(todos_resultados)
print("TABLA RESUMEN - ANÁLISIS DE DISPERSIÓN")
print("=====================================")
print(tabla_final)

[1] "TABLA RESUMEN - ANÁLISIS DE DISPERSIÓN"
[1] "====================================="
  Cobertura    D_j P_valor_Cameron_Trivedi   Distribucion_Recomendada
1       PPD 1.3357                   0e+00 Borderline - revisar tests
2       PPH 0.6800                   1e-04                Subdispersa
3       PTH 0.6975                   3e-04                Subdispersa
4        RC 1.4244                   0e+00 Borderline - revisar tests
